<h1>Maritime: Weather</h1>

In [1]:
import pandas as pd
import numpy as np
from math import atan2, degrees
from datetime import datetime
import xarray as xr
from pathlib import Path

# PATHS
AIS_COLLISION_CSV = "collision/ais raw data/gfw_encounters_recent.csv"
AIS_WEATHER_CSV = "weather/ais_weather.csv"
AIS_WEATHERAPI_CSV = "weather/ais_weatherapi.csv"

WIND_NC = "weather/wind10m.nc"
WAVES_NC = "weather/waves_subset.nc"
CURRENTS_NC = "weather/currents_subset.nc"

<h2>Retrieving Raw AIS Data Used In Collision to call Weather API to get data</h2>

In [2]:
df = pd.read_csv(AIS_COLLISION_CSV)

# --- choose timestamp ---
df["ts_utc"] = pd.to_datetime(df["start"], errors="coerce").dt.strftime("%Y-%m-%dT%H:%M:%SZ")

# --- lat/lon (prefer explicit columns; fallback to boundingBox) ---
def parse_bbox(b):
    # expected like "[lat1, lon1, lat2, lon2]" or "[lon, lat, lon, lat]" depending on source!
    try:
        vals = [float(x) for x in str(b).strip("[]").split(",")]
        if len(vals) >= 2:
            # Heuristic: if |vals[0]| <= 90 and |vals[1]| <= 180, assume [lat, lon]
            if abs(vals[0]) <= 90 and abs(vals[1]) <= 180:
                return pd.Series({"lat_fallback": vals[0], "lon_fallback": vals[1]})
            else:
                # else assume [lon, lat]
                return pd.Series({"lat_fallback": vals[1], "lon_fallback": vals[0]})
    except Exception:
        pass
    return pd.Series({"lat_fallback": np.nan, "lon_fallback": np.nan})

bbox_fallback = df["boundingBox"].apply(parse_bbox)
df = pd.concat([df, bbox_fallback], axis=1)

df["lat"] = df["position__lat"].where(df["position__lat"].notna(), df["lat_fallback"])
df["lon"] = df["position__lon"].where(df["position__lon"].notna(), df["lon_fallback"])

# --- sog ---
df["sog"] = pd.to_numeric(df["encounter__medianSpeedKnots"], errors="coerce").round(1)

# --- mmsi from ssvid (keep only 9-digit numeric if you want strict MMSI) ---
def normalize_mmsi(x):
    s = str(x).strip()
    return s if s.isdigit() and len(s) == 9 else np.nan

df["mmsi"] = df["vessel__ssvid"].apply(normalize_mmsi)

# --- vessel_type ---
df["vessel_type"] = df["vessel__type"].fillna("Unknown")

# --- compute COG by bearing to next point per vessel ---
# Sort by vessel + time
df["_t"] = pd.to_datetime(df["ts_utc"], errors="coerce")
df = df.sort_values(["mmsi", "_t"])

# Bearing helper
def bearing_deg(lat1, lon1, lat2, lon2):
    if any(pd.isna([lat1, lon1, lat2, lon2])):
        return np.nan
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    y = np.sin(dlon) * np.cos(lat2)
    x = np.cos(lat1)*np.sin(lat2) - np.sin(lat1)*np.cos(lat2)*np.cos(dlon)
    brng = (degrees(atan2(y, x)) + 360) % 360
    return brng

# Next point within same vessel
df["lat_next"] = df.groupby("mmsi")["lat"].shift(-1)
df["lon_next"] = df.groupby("mmsi")["lon"].shift(-1)

df["cog"] = df.apply(lambda r: bearing_deg(r["lat"], r["lon"], r["lat_next"], r["lon_next"]), axis=1)
df["cog"] = df["cog"].round(0).astype("Int64").astype(str).str.zfill(3)
df.loc[df["cog"]=="<NA>", "cog"] = ""

# --- final selection & cleaning ---
out = df[["ts_utc", "lat", "lon", "sog", "cog", "mmsi", "vessel_type"]].copy()
out = out.dropna(subset=["ts_utc", "lat", "lon"])
out["lat"] = out["lat"].round(3)
out["lon"] = out["lon"].round(3)

# save
out.to_csv(AIS_WEATHER_CSV, index=False)
print(out.head(5).to_string(index=False))


              ts_utc    lat     lon  sog cog      mmsi vessel_type
2025-03-21T05:00:00Z 36.947 123.323  1.3     100040506     fishing
2025-03-19T15:20:00Z 30.824 123.253  1.9 039 100106189     fishing
2025-03-21T04:20:00Z 30.849 123.276  1.9     100106189     fishing
2025-03-20T14:40:00Z 28.315 123.851  0.0     100222222     fishing
2025-03-20T10:40:00Z 22.356 117.581  0.0 281 100305237     fishing


<h2>Calling copernicusmarine API and wind.f000 to get relevant weather data</h2>

In [3]:
def sample_vars_to_points(nc_path, ais_df, wanted):
    """
    Samples variables from an xarray dataset at AIS (time, lat, lon) points.
    - Works with 1-D or 2-D lat/lon.
    - Uses time selection only if there is a real time dimension; otherwise skips time.
    """
    if not nc_path or nc_path == "-" or not Path(nc_path).exists():
        return {}

    ds = xr.open_dataset(nc_path)

    # ---- find lat/lon names ----
    lat_name = next((n for n in ["latitude","lat","nav_lat"] if n in ds.variables or n in ds.coords), None)
    lon_name = next((n for n in ["longitude","lon","nav_lon"] if n in ds.variables or n in ds.coords), None)
    if not lat_name or not lon_name:
        raise SystemExit(f"Could not find lat/lon in coords={list(ds.coords)} vars={list(ds.data_vars)}")

    # ---- collapse 2-D lat/lon (y,x) to 1-D if separable ----
    latn, lonn = None, None
    if getattr(ds[lat_name], "ndim", 1) == 2 and "y" in ds.dims and "x" in ds.dims:
        lat2d = ds[lat_name].values
        lon2d = ds[lon_name].values
        lat1d = lat2d[:, 0]
        lon1d = lon2d[0, :]
        if np.allclose(lat2d, lat1d[:, None], equal_nan=True) and np.allclose(lon2d, lon1d[None, :], equal_nan=True):
            ds = ds.assign_coords(y=("y", lat1d), x=("x", lon1d))
            latn, lonn = "y", "x"
        else:
            raise SystemExit(
                "Field has non-separable 2D lat/lon. Re-export as regular 1D latitude/longitude NetCDF."
            )

    # Pick 1-D coord names (normal case or after collapsing)
    def pick1d(ds, cands):
        for c in cands:
            if (c in ds.coords or c in ds.dims) and getattr(ds[c], "ndim", 1) == 1:
                return c
        return None

    if latn is None: latn = pick1d(ds, ["latitude","lat","y"])
    if lonn is None: lonn = pick1d(ds, ["longitude","lon","x"])
    if latn is None or lonn is None:
        raise SystemExit(f"No 1-D lat/lon coords. Coords: {list(ds.coords)} | Dims: {list(ds.dims)}")

    # ---- time handling ----
    tdim = None
    for cand in ("time","valid_time","step"):
        if cand in ds.dims and getattr(ds[cand], "ndim", 1) == 1:
            tdim = cand
            break

    # If only one time slice, drop time dim
    if tdim and ds.dims.get(tdim, 0) == 1:
        ds = ds.isel({tdim: 0})
        tdim = None

    # keep only variables we actually have
    avail = [v for v in wanted if v in ds.variables]
    if not avail:
        return {}

    # sort coords if 1-D
    for c in ([tdim] if tdim else []) + [latn, lonn]:
        if c and c in ds.coords and getattr(ds[c], "ndim", 1) == 1:
            ds = ds.sortby(c)

    # build selection target
    target = {
        latn: xr.DataArray(ais_df["lat"].values, dims="points"),
        lonn: xr.DataArray(ais_df["lon"].values, dims="points"),
    }
    if tdim:
        target[tdim] = xr.DataArray(ais_df["ts_utc"].values, dims="points")

    sampled = ds[avail].sel(target, method="nearest")
    return {v: sampled[v].values for v in avail}


In [4]:
# === Load AIS FROM AIS_WEATHER_CSV and normalize ===

# Expecting columns at least: ts_utc, lat, lon
# Optional but recommended: sog, cog (degrees 0..359)
ais = pd.read_csv(AIS_WEATHER_CSV)

# Ensure columns exist or map common variants
rename_map = {}
if "timestamp" in ais.columns: rename_map["timestamp"] = "ts_utc"
if "Latitude"  in ais.columns: rename_map["Latitude"]  = "lat"
if "Longitude" in ais.columns: rename_map["Longitude"] = "lon"
if "SOG"       in ais.columns: rename_map["SOG"]       = "sog"
if "COG"       in ais.columns: rename_map["COG"]       = "cog"
ais = ais.rename(columns=rename_map)

# Required columns check
for col in ("ts_utc","lat","lon"):
    if col not in ais.columns:
        raise SystemExit(f"Missing AIS column: {col}")

# Parse time -> naive UTC (ns precision ok for xarray)
ais["ts_utc"] = pd.to_datetime(ais["ts_utc"], errors="coerce", utc=True).dt.tz_convert(None)

# Coerce numerics and clean
for c in ("lat","lon","sog"):
    if c in ais.columns:
        ais[c] = pd.to_numeric(ais[c], errors="coerce")

# COG: keep as numeric degrees for trig later
if "cog" in ais.columns:
    ais["cog"] = pd.to_numeric(ais["cog"], errors="coerce")


In [5]:
# === Sample environmental fields ===

waves = sample_vars_to_points(WAVES_NC,    ais, wanted=["VHM0","VMDR","VTM10","VTPK"])
currs = sample_vars_to_points(CURRENTS_NC, ais, wanted=["uo","vo","eastward_current","northward_current"])
wind  = sample_vars_to_points(WIND_NC,     ais, wanted=["u10","v10","UGRD_10maboveground","VGRD_10maboveground"])

out = ais.copy()

# Attach samples if present
for k, v in waves.items(): out[k] = v
for k, v in currs.items(): out[k] = v
for k, v in wind.items():  out[k] = v

# Standardize aliases
if "UGRD_10maboveground" in out.columns: out = out.rename(columns={"UGRD_10maboveground":"u10"})
if "VGRD_10maboveground" in out.columns: out = out.rename(columns={"VGRD_10maboveground":"v10"})
if "eastward_current"    in out.columns: out = out.rename(columns={"eastward_current":"uo"})
if "northward_current"   in out.columns: out = out.rename(columns={"northward_current":"vo"})


C:\Users\zepha\AppData\Local\Temp\ipykernel_34260\2802190509.py:53: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if tdim and ds.dims.get(tdim, 0) == 1:
<frozen _collections_abc>:807: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
C:\Users\zepha\AppData\Local\Temp\ipykernel_34260\2802190509.py:53: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if tdim and ds.dims.get(tdim, 0) == 1:
<frozen _coll

In [6]:
# === Derived features ===

# Waves
if "VHM0" in out.columns:
    out["stormy"] = (out["VHM0"] >= 3.0).astype(int)

# Relative wave angle / sector (needs VMDR and COG)
if {"VMDR","cog"}.issubset(out.columns):
    # VMDR = mean wave direction (from-direction). Convert to “to” direction to compare with COG.
    wave_to = (out["VMDR"] + 180) % 360
    ang = np.abs((wave_to - out["cog"]) % 360)
    ang = np.where(ang > 180, 360 - ang, ang)
    out["rel_wave_angle"] = ang
    out["sea_sector"] = pd.cut(
        out["rel_wave_angle"],
        bins=[-0.1, 30, 150, 180],
        labels=["following", "beam", "head"]
    )

# Currents projection into along/cross relative to COG
if {"uo","vo","cog"}.issubset(out.columns):
    theta = np.deg2rad(out["cog"])
    hx, hy = np.sin(theta), np.cos(theta)  # heading unit vector (x=east, y=north)
    along_ms = out["uo"] * hx + out["vo"] * hy
    cross_ms = -out["uo"] * hy + out["vo"] * hx
    out["along_current_kn"] = along_ms * 1.94384
    out["cross_current_kn"] = cross_ms * 1.94384
    if "sog" in out.columns:
        out["stw_est_kn"] = out["sog"] - out["along_current_kn"]  # crude STW estimate

# Wind speed/dir and relative wind angle
if {"u10","v10"}.issubset(out.columns):
    out["wind_speed_ms"] = np.hypot(out["u10"], out["v10"])
    # “wind-to” direction (0=N, 90=E) using arctan2(u_east, v_north)
    out["wind_dir_to"] = (np.degrees(np.arctan2(out["u10"], out["v10"])) + 360) % 360
    if "cog" in out.columns:
        wang = (out["wind_dir_to"] - out["cog"]) % 360
        out["rel_wind_angle"] = np.where(wang > 180, 360 - wang, wang)


In [7]:
# === Save to your desired output ===
out.to_csv(AIS_WEATHERAPI_CSV, index=False)
print("Wrote", AIS_WEATHERAPI_CSV, "with shape", out.shape)

Wrote weather/ais_weatherapi.csv with shape (8000, 23)


<h2>Checking new csv file</h2>

In [8]:
df = pd.read_csv(AIS_WEATHERAPI_CSV)

print(df.shape, df.columns.tolist())
df.describe(include='all').T.head(20)

# essential fields present
assert {"ts_utc","lat","lon"}.issubset(df.columns)
# lat/lon sanity
assert df["lat"].between(-90,90).all()
assert df["lon"].between(-180,180).all()
df.isna().mean().sort_values(ascending=False).head(15)


(8000, 23) ['ts_utc', 'lat', 'lon', 'sog', 'cog', 'mmsi', 'vessel_type', 'VHM0', 'VMDR', 'VTM10', 'uo', 'vo', 'u10', 'v10', 'stormy', 'rel_wave_angle', 'sea_sector', 'along_current_kn', 'cross_current_kn', 'stw_est_kn', 'wind_speed_ms', 'wind_dir_to', 'rel_wind_angle']


sea_sector          0.791500
along_current_kn    0.791500
rel_wave_angle      0.791500
stw_est_kn          0.791500
cross_current_kn    0.791500
VHM0                0.468250
uo                  0.468250
vo                  0.468250
VTM10               0.468250
VMDR                0.468000
rel_wind_angle      0.360000
cog                 0.360000
mmsi                0.018375
ts_utc              0.000000
lon                 0.000000
dtype: float64

In [9]:
# time and sort
df["ts_utc"] = pd.to_datetime(df["ts_utc"], errors="coerce", utc=True).dt.tz_convert(None)
df = df.sort_values(["ts_utc","lat","lon"]).drop_duplicates(subset=["ts_utc","lat","lon"])


In [10]:
df.head()

,ts_utc,lat,lon,sog,cog,mmsi,vessel_type,VHM0,VMDR,VTM10,...,v10,stormy,rel_wave_angle,sea_sector,along_current_kn,cross_current_kn,stw_est_kn,wind_speed_ms,wind_dir_to,rel_wind_angle
5622,2025-03-08 06:20:00,19.502,57.741,0.0,328.0,412549025.0,fishing,NaN,NaN,NaN,...,5.202881,0,NaN,NaN,NaN,NaN,NaN,5.479104,18.27066,50.270660
202,2025-03-10 05:30:00,19.481,57.730,0.0,90.0,271072244.0,fishing,NaN,NaN,NaN,...,5.202881,0,NaN,NaN,NaN,NaN,NaN,5.479104,18.27066,71.729340
310,2025-03-10 20:00:00,48.393,-64.493,0.0,198.0,316004122.0,fishing,NaN,NaN,NaN,...,0.762881,0,NaN,NaN,NaN,NaN,NaN,5.243527,81.63434,116.365662
1292,2025-03-10 20:40:00,26.548,50.032,0.0,315.0,403772940.0,fishing,NaN,NaN,NaN,...,-3.787119,0,NaN,NaN,NaN,NaN,NaN,3.800806,184.86389,130.136108
5012,2025-03-11 04:40:00,-1.385,-104.911,1.0,NaN,412440150.0,fishing,NaN,NaN,NaN,...,5.612881,0,NaN,NaN,NaN,NaN,NaN,5.741346,12.14328,NaN


In [11]:
df.describe()

,ts_utc,lat,lon,sog,cog,mmsi,VHM0,VMDR,VTM10,uo,...,u10,v10,stormy,rel_wave_angle,along_current_kn,cross_current_kn,stw_est_kn,wind_speed_ms,wind_dir_to,rel_wind_angle
count,2369,2369.000000,2369.000000,2369.000000,1050.000000,2.351000e+03,1898.000000,1899.000000,1898.000000,1898.000000,...,2369.000000,2369.000000,2369.0,702.000000,702.000000,702.000000,702.000000,2369.000000,2369.000000,1050.000000
mean,2025-03-20 14:28:38.446601728,23.904895,102.055734,0.473913,182.115238,4.290243e+08,0.518730,307.855942,6.837018,-0.129005,...,-0.446364,1.637305,0.0,92.605954,0.002017,0.012689,0.298838,5.282671,164.743279,88.567380
min,2025-03-08 06:20:00,-60.699000,-176.036000,0.000000,0.000000,1.000405e+08,0.190000,114.910001,4.410000,-0.234490,...,-10.372266,-8.617119,0.0,0.130003,-1.102532,-0.253689,-0.291710,0.731341,0.188416,0.045074
25%,2025-03-20 09:20:00,19.096000,106.661000,0.000000,93.000000,4.037869e+08,0.520000,309.869997,6.860000,-0.130368,...,-2.172265,-0.587119,0.0,43.130003,-0.168575,-0.160046,-0.116709,3.800806,54.611480,44.316803
50%,2025-03-20 19:50:00,26.547000,115.254000,0.000000,180.000000,4.124224e+08,0.520000,309.869997,6.860000,-0.130368,...,-0.202266,1.242881,0.0,100.130003,0.012300,0.012300,0.134808,5.070917,184.863890,88.004456
75%,2025-03-21 04:00:00,32.047000,122.250000,0.900000,283.000000,4.310102e+08,0.520000,309.869997,6.860000,-0.130368,...,1.847734,5.222881,0.0,135.065001,0.190835,0.196161,0.287700,6.487960,274.222530,132.317955
max,2025-03-21 13:30:00,67.492000,175.829000,2.000000,360.000000,9.980878e+08,0.710000,309.869997,6.860000,0.184563,...,15.727735,8.882881,0.0,179.869997,0.291710,1.104952,2.147963,16.781473,359.586240,179.673920
std,NaN,13.072287,38.867878,0.670426,107.013987,1.130295e+08,0.022011,17.737083,0.207264,0.017349,...,3.928948,3.649354,0.0,52.158739,0.184412,0.184310,0.601006,1.928015,120.141786,52.212665


In [ ]:
# prints the number of null values in each column
null_counts = df.isnull().sum()
print(null_counts)

ts_utc                 0
lat                    0
lon                    0
sog                    0
cog                 1319
mmsi                  18
vessel_type            0
VHM0                 471
VMDR                 470
VTM10                471
uo                   471
vo                   471
u10                    0
v10                    0
stormy                 0
rel_wave_angle      1667
sea_sector          1667
along_current_kn    1667
cross_current_kn    1667
stw_est_kn          1667
wind_speed_ms          0
wind_dir_to            0
rel_wind_angle      1319
dtype: int64


In [14]:
df.shape

(2369, 23)

In [ ]:
#boxplot

In [ ]:
#filling na values with median
df_clean = df
df_clean["cog"].fillna(df["cog"].median(), inplace=True)
df_clean["VHM0"].fillna(df["VHM0"].median(), inplace=True)
df_clean["VTM10"].fillna(df["VTM10"].median(), inplace=True)
df_clean["uo"].fillna(df["uo"].median(), inplace=True)
df_clean["vo"].fillna(df["vo"].median(), inplace=True)
df_clean["along_current_kn"].fillna(df["along_current_kn"].median(), inplace=True)
df_clean["cross_current_kn"].fillna(df["cross_current_kn"].median(), inplace=True)
df_clean["stw_est_kn"].fillna(df["stw_est_kn"].median(), inplace=True)
# df_clean["mmsi"].fillna(df["mmsi"].median(), inplace=True)
# df_clean["VMDR"].fillna(df["VMDR"].median(), inplace=True)
# df_clean["rel_wave_angle"].fillna(df["rel_wave_angle"].median(), inplace=True)
# df_clean["sea_sector"].fillna(df["sea_sector"].median(), inplace=True)
# df_clean["rel_wind_angle"].fillna(df["rel_wind_angle"].median(), inplace=True)
null_counts = df_clean.isnull().sum()
print(null_counts)    

ts_utc                 0
lat                    0
lon                    0
sog                    0
cog                    0
mmsi                  18
vessel_type            0
VHM0                   0
VMDR                   0
VTM10                  0
uo                     0
vo                     0
u10                    0
v10                    0
stormy                 0
rel_wave_angle      1667
sea_sector          1667
along_current_kn       0
cross_current_kn       0
stw_est_kn             0
wind_speed_ms          0
wind_dir_to            0
rel_wind_angle      1319
dtype: int64


C:\Users\zepha\AppData\Local\Temp\ipykernel_34260\2156015717.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean["cog"].fillna(df["cog"].median(), inplace=True)
C:\Users\zepha\AppData\Local\Temp\ipykernel_34260\2156015717.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


In [19]:
df_clean.head()

,ts_utc,lat,lon,sog,cog,mmsi,vessel_type,VHM0,VMDR,VTM10,...,v10,stormy,rel_wave_angle,sea_sector,along_current_kn,cross_current_kn,stw_est_kn,wind_speed_ms,wind_dir_to,rel_wind_angle
5622,2025-03-08 06:20:00,19.502,57.741,0.0,328.0,412549025.0,fishing,0.52,309.869997,6.86,...,5.202881,0,NaN,NaN,0.0123,0.0123,0.134808,5.479104,18.27066,50.270660
202,2025-03-10 05:30:00,19.481,57.730,0.0,90.0,271072244.0,fishing,0.52,309.869997,6.86,...,5.202881,0,NaN,NaN,0.0123,0.0123,0.134808,5.479104,18.27066,71.729340
310,2025-03-10 20:00:00,48.393,-64.493,0.0,198.0,316004122.0,fishing,0.52,309.869997,6.86,...,0.762881,0,NaN,NaN,0.0123,0.0123,0.134808,5.243527,81.63434,116.365662
1292,2025-03-10 20:40:00,26.548,50.032,0.0,315.0,403772940.0,fishing,0.52,309.869997,6.86,...,-3.787119,0,NaN,NaN,0.0123,0.0123,0.134808,3.800806,184.86389,130.136108
5012,2025-03-11 04:40:00,-1.385,-104.911,1.0,180.0,412440150.0,fishing,0.52,309.869997,6.86,...,5.612881,0,NaN,NaN,0.0123,0.0123,0.134808,5.741346,12.14328,NaN


In [21]:
#one-hot encoding
df_clean_encoded = pd.get_dummies(df_clean, columns=["vessel_type", "sea_sector"], drop_first=True)
df_clean_encoded.head()

,ts_utc,lat,lon,sog,cog,mmsi,VHM0,VMDR,VTM10,uo,...,rel_wave_angle,along_current_kn,cross_current_kn,stw_est_kn,wind_speed_ms,wind_dir_to,rel_wind_angle,vessel_type_fishing,sea_sector_following,sea_sector_head
5622,2025-03-08 06:20:00,19.502,57.741,0.0,328.0,412549025.0,0.52,309.869997,6.86,-0.130368,...,NaN,0.0123,0.0123,0.134808,5.479104,18.27066,50.270660,True,False,False
202,2025-03-10 05:30:00,19.481,57.730,0.0,90.0,271072244.0,0.52,309.869997,6.86,-0.130368,...,NaN,0.0123,0.0123,0.134808,5.479104,18.27066,71.729340,True,False,False
310,2025-03-10 20:00:00,48.393,-64.493,0.0,198.0,316004122.0,0.52,309.869997,6.86,-0.130368,...,NaN,0.0123,0.0123,0.134808,5.243527,81.63434,116.365662,True,False,False
1292,2025-03-10 20:40:00,26.548,50.032,0.0,315.0,403772940.0,0.52,309.869997,6.86,-0.130368,...,NaN,0.0123,0.0123,0.134808,3.800806,184.86389,130.136108,True,False,False
5012,2025-03-11 04:40:00,-1.385,-104.911,1.0,180.0,412440150.0,0.52,309.869997,6.86,-0.130368,...,NaN,0.0123,0.0123,0.134808,5.741346,12.14328,NaN,True,False,False
